In [2]:
import gzip

In [32]:
count = 0
with gzip.open('freebase_douban.gz', 'rb') as f:
    for line in f:
        line = line.strip()
        triplet = line.decode().split('\t')[:3]
        print(triplet)
        count += 1
        if count == 20:
            break

['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/m.04n2x3p>']
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/m.058y23j>']
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/m.07gb1q>']
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/g.125_c52px>']
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/m.0d91lf>']
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns/m.04hk1c>']
['<http://rdf.freebase.com/ns/award.award_winner>', '<http://rdf.freebase.com/ns/type.type.instance>', '<http://rdf.freebase.com/ns

In [9]:
id2entity = {}
movie_set = set()

with open('douban2fb.txt', "r", encoding='utf-8') as f:
    for line in f.readlines():
        id, entity = line.strip().split()
        id2entity[id] = "<http://rdf.freebase.com/ns/" + entity + ">"
        movie_set.add("<http://rdf.freebase.com/ns/" + entity + ">")

print(id2entity)

{'1291544': '<http://rdf.freebase.com/ns/m.03177r>', '1291545': '<http://rdf.freebase.com/ns/m.027pfg>', '1291546': '<http://rdf.freebase.com/ns/m.01d1_s>', '1291550': '<http://rdf.freebase.com/ns/m.053xlz>', '1291552': '<http://rdf.freebase.com/ns/m.017jd9>', '1291554': '<http://rdf.freebase.com/ns/m.01sxdy>', '1291555': '<http://rdf.freebase.com/ns/m.01f8f7>', '1291557': '<http://rdf.freebase.com/ns/m.01f85k>', '1291558': '<http://rdf.freebase.com/ns/m.04j31dn>', '1291559': '<http://rdf.freebase.com/ns/m.05_lhx>', '1291561': '<http://rdf.freebase.com/ns/m.0j6b5>', '1291562': '<http://rdf.freebase.com/ns/m.01m13b>', '1291565': '<http://rdf.freebase.com/ns/m.02dglg>', '1291571': '<http://rdf.freebase.com/ns/m.017gl1>', '1291573': '<http://rdf.freebase.com/ns/m.02c7k4>', '1291574': '<http://rdf.freebase.com/ns/m.01xdxy>', '1291575': '<http://rdf.freebase.com/ns/m.0dyb1>', '1291577': '<http://rdf.freebase.com/ns/m.02qm_f>', '1291578': '<http://rdf.freebase.com/ns/m.06w839_>', '1291579': 

In [34]:
with gzip.open('freebase_douban.gz', 'rb') as f:
    with open('extract_entity', "a", encoding = 'utf-8') as f1: 
        for line in f:
            line = line.strip()
            triplet = line.decode().strip().split('\t')[:3]
            if triplet[2] in movie_set or triplet[0] in movie_set:
                f1.write("\t".join(line.decode().strip().split('\t')[:3]) + "\n")


In [11]:
entity_cnt = {} #记录实体出现了多少次
relation_cnt = {} #记录关系出现了多少次
relation_set = set() #记录所有超过50条的关系
new_entity_set = set()  #表示扩充后的实体集合


pre_row = []  #表示按前缀过滤后的行
with open('extract_entity', "r", encoding='utf-8') as f:
    for line in f.readlines():
        triplet = line.strip().split('\t')
        if(triplet[0].startswith("<http://rdf.freebase.com/ns/") and triplet[2].startswith("<http://rdf.freebase.com/ns/")):
            pre_row.append(line)

for line in pre_row:  #统计关系出现次数
    triplet = line.strip().split('\t')
    if(triplet[1] not in relation_cnt):
        relation_cnt[triplet[1]] = 1
    else:
        relation_cnt[triplet[1]] += 1

relation_row = [] #表示按关系数量过滤后的行
for line in pre_row:
    triplet = line.strip().split('\t')
    if(relation_cnt[triplet[1]] > 50):
        relation_row.append(line)

for line in relation_row:   #统计过滤后的行中各实体出现次数
    triplet = line.strip().split('\t')
    if(triplet[0] not in entity_cnt):
        entity_cnt[triplet[0]] = 1
    else:
        entity_cnt[triplet[0]] += 1
    if(triplet[0] != triplet[2]):
        if(triplet[2] not in entity_cnt):
            entity_cnt[triplet[2]] = 1
        else:
            entity_cnt[triplet[2]] += 1

for key in entity_cnt:      #过滤实体
    if(entity_cnt[key] > 20):
        new_entity_set.add(key)
'''
with open('extract_entity', "r", encoding='utf-8') as f:
    for line in f.readlines():
        triplet = line.strip().split('\t')
        if(triplet[1] not in relation_cnt):
            relation_cnt[triplet[1]] = 1
        else:
            relation_cnt[triplet[1]] += 1
            if(relation_cnt[triplet[1]] > 50 and (triplet[1] not in relation_set)):
                relation_set.add(triplet[1])
        
    for line in f.readlines():
        triplet = line.strip().split('\t')
        if(triplet[1] in relation_set):
            if(not triplet[0].startswith("<http://rdf.freebase.com/ns/") or not triplet[2].startswith("<http://rdf.freebase.com/ns/")):
                relation_set.remove(triplet[1])

with open('extract_entity', "r", encoding='utf-8') as f:
    for line in f.readlines():
        triplet = line.strip().split('\t')
        if(triplet[1] in relation_set ): #若该关系是保留关系
            if(triplet[0] not in entity_cnt):
                entity_cnt[triplet[0]] = 1
            else:
                entity_cnt[triplet[0]] += 1
            if(triplet[2] not in entity_cnt):
                entity_cnt[triplet[2]] = 1
            else:
                entity_cnt[triplet[2]] += 1
                

with open('extract_entity', "r", encoding='utf-8') as f:
    with open('KG_result', "a", encoding='utf-8') as f1:
        for line in f.readlines():
            triplet = line.strip().split('\t')
            if(triplet[1] in relation_set and entity_cnt[triplet[0]] >= 20 and entity_cnt[triplet[2]] >= 20):
                f1.write("\t".join(line.strip().split('\t')[:3]) + "\n") #写入图谱
                if(triplet[0] not in new_entity_set):
                    new_entity_set.add(triplet[0])
                if(triplet[2] not in new_entity_set):
                    new_entity_set.add(triplet[2])
'''
print(len(new_entity_set))
#print(entity_cnt)
#print("\n")
#print(relation_cnt)

885


In [57]:
for element in movie_set:   #检验：原movie实体均在扩充后的集合中
    if(element not in new_entity_set):
        print("error")


In [59]:
'''
with open('KG_result', "a", encoding='utf-8') as f:
    for line in relation_row:   
        triplet = line.strip().split('\t')
        if(triplet[0] in new_entity_set and triplet[2] in new_entity_set):
            f.write("\t".join(line.strip().split('\t')[:3]) + "\n") #写入图谱
'''

In [60]:
with gzip.open('freebase_douban.gz', 'rb') as f:
    with gzip.open('extract_entity_new.gz', "wt", encoding = 'utf-8') as f1: 
        for line in f:
            line = line.strip()
            triplet = line.decode().strip().split('\t')[:3]
            if(triplet[2] in new_entity_set or triplet[0] in new_entity_set):
                f1.write("\t".join(line.decode().strip().split('\t')[:3]) + "\n")  

In [4]:
entity_cnt = {} #记录实体出现了多少次
relation_cnt = {} #记录关系出现了多少次
relation_set = set() #记录所有超过50条的关系
new_entity_set = set()  #表示扩充后的实体集合

entity_cnt_first = {} #记录实体出现了多少次(初筛)
relation_cnt_first = {} #记录关系出现了多少次（初筛）
with gzip.open('extract_entity_new.gz', "rb") as f:
    for line in f:
        line = line.strip()
        triplet = line.decode().strip().split('\t')[:3]
        if(triplet[1] not in relation_cnt_first):
            relation_cnt_first[triplet[1]] = 1
        else:
            relation_cnt_first[triplet[1]] += 1
        if(triplet[0] not in entity_cnt_first):
            entity_cnt_first[triplet[0]] = 1
        else:
            entity_cnt_first[triplet[0]] += 1
        if(triplet[2] not in entity_cnt_first):
            entity_cnt_first[triplet[2]] = 1
        else:
            entity_cnt_first[triplet[2]] += 1


In [5]:
pre_row = []  #表示按前缀过滤后的行
with gzip.open('extract_entity_new.gz', "rb") as f:
    for line in f:
        line = line.strip()
        triplet = line.decode().strip().split('\t')[:3]
        if(relation_cnt_first[triplet[1]] >= 50):
            if(entity_cnt_first[triplet[0]] <= 20000 and entity_cnt_first[triplet[2]] <= 20000):
                if(triplet[0].startswith("<http://rdf.freebase.com/ns/") and triplet[2].startswith("<http://rdf.freebase.com/ns/")):
                    pre_row.append(line.decode())

In [12]:
print(len(pre_row))

240073


In [6]:
for line in pre_row:  #统计关系出现次数
    triplet = line.strip().split('\t')
    if(triplet[1] not in relation_cnt):
        relation_cnt[triplet[1]] = 1
    else:
        relation_cnt[triplet[1]] += 1

relation_row = [] #表示按关系数量过滤后的行
for line in pre_row:
    triplet = line.strip().split('\t')
    if(relation_cnt[triplet[1]] > 50):
        relation_row.append(line)

for line in relation_row:   #统计过滤后的行中各实体出现次数
    triplet = line.strip().split('\t')
    if(triplet[0] not in entity_cnt):
        entity_cnt[triplet[0]] = 1
    else:
        entity_cnt[triplet[0]] += 1
    if(triplet[2] not in entity_cnt):
        entity_cnt[triplet[2]] = 1
    else:
        entity_cnt[triplet[2]] += 1

for key in entity_cnt:      #过滤实体
    if(entity_cnt[key] > 15 and entity_cnt[key] < 20000):
        new_entity_set.add(key)
print(len(new_entity_set))
#print(entity_cnt)
#print("\n")
#print(relation_cnt)

8892


In [7]:
with open('KG_result.txt', "w", encoding='utf-8') as f:
    for line in relation_row:   
        triplet = line.strip().split('\t')
        if(triplet[0] in new_entity_set and triplet[2] in new_entity_set):
            f.write("\t".join(line.strip().split('\t')[:3]) + "\n") #写入图谱

In [10]:
for element in movie_set:   #检验：原movie实体均在扩充后的集合中
    if(element not in new_entity_set):
        print("error")